In [ ]:
import torch
import time
import pandas as pd

In [ ]:
def timer(func): # timer using decorator
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    computation_time = end_time - start_time
    print(f"Execution time of {func.__name__}: {computation_time} seconds")
    return result
  return wrapper

In [ ]:
data = pd.read_pickle('quiz_data.pkl')
# print(data['x'][1])
N = len(data['x'])
idx = list(range(N))
split_index = int(N * 0.8)

train_idx = idx[:split_index]
val_idx = idx[split_index:]

x_train, y_train = data['x'][train_idx], data['y'][train_idx] #trainset과 trainset의 label생성
x_val, y_val = data['x'][val_idx], data['y'][val_idx]

In [ ]:
#create tensor at CPU #tensor는 3차원 이상의 자료구조(매트릭스 이상의)
x_train_tensor = torch.as_tensor(data['x'])
y_train_tensor = torch.as_tensor(data['y'])

#create tensor at GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_train_tensor = torch.as_tensor(x_train).to(device)
y_train_tensor = torch.as_tensor(y_train).to(device)

In [ ]:
@timer
def train_model_torch(lr=0.1, epochs=10):
  b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)
  mse_loss_sum = 0
  for epoch in range(epochs):
    #Loss computation
    y_hat = b + w * x_train_tensor
    error = (y_hat - y_train_tensor)
    mse_loss = torch.mean(error ** 2)
    mse_loss_sum += mse_loss
    #gradient computation and descent
    mse_loss.backward()
    with torch.no_grad():
      b -= lr * b.grad
      w -= lr * w.grad
    b.grad.zero_()
    w.grad.zero_()
    print(mse_loss_sum/epochs)
  return w, b

In [ ]:
train_model_torch()

UnboundLocalError: ignored

In [ ]:
from torch import nn, optim

In [ ]:
@timer
def train_model_optim(lr=0.1, epochs=10):
  b = torch.randn(1, requires_grad=True, dtype=torch.float, device=device) #requires_grad means this param is
  w = torch.randn(1, requires_grad=True, dtype=torch.float, device=device)

  parameters = [b, w]
  optimizer = optim.SGD(parameters, lr = lr)
  mse_loss = nn.MSELoss()
  for epoch in range(epochs):
    #Loss computation
    y_hat = b + w * x_train_tensor
    loss = mse_loss(y_hat, y_train_tensor)
    #gradient computation and descent
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  last_y_hat = b + w * x_train_tensor
  last_error = (last_y_hat - y_train_tensor)
  last_mse_loss = torch.mean(last_error ** 2)
  print(last_mse_loss)


  return w, b

In [ ]:
train_w, train_b = train_model_optim()
train_w, train_b

tensor(0.3926, dtype=torch.float64, grad_fn=<MeanBackward0>)
Execution time of train_model_optim: 0.00994110107421875 seconds


(tensor([0.6934], requires_grad=True), tensor([-0.6829], requires_grad=True))